In [1]:
from datasets import Dataset
import pandas as pd
import numpy as np
!pip install datasets --quiet
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

In [2]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"
!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

--2024-06-08 20:48:23--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.03s   

2024-06-08 20:48:24 (64.0 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-06-08 20:48:25--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [52]:
train = pd.read_table("/kaggle/working/train.tsv")
# train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)

trial = pd.read_table("/kaggle/working/trial.tsv")
# trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)

train_data = train[:-100]
trial = trial[:35].reset_index(drop = True)
from datasets import Dataset

train_data = Dataset.from_pandas(train_data)
# eval_data = Dataset.from_pandas(eval_data)
eval_data = Dataset.from_pandas(trial)

In [53]:
def remove_quotations(text):
    # Remove both double and single quotation marks
    text = text.replace('"', '')  # Remove double quotations
    text = text.replace("'", "")  # Remove single quotations
    return text

def preprocess_function(examples):
    # Apply remove_quotations function to both sentences and tokens
    sentences = [remove_quotations(f"{s} [SEP] {t}") for s, t in zip(examples['sentence'], examples['token'])]
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=64)
    tokenized_inputs['labels'] = examples['complexity']
    return tokenized_inputs

In [54]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# Map the updated preprocess function to datasets
train_dataset = train_data.map(preprocess_function, batched=True, batch_size=64)
eval_dataset = eval_data.map(preprocess_function, batched=True, batch_size=16)

Map:   0%|          | 0/7132 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [51]:
from transformers import BertConfig, BertTokenizerFast, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_path = 'bert-base-uncased'

config = BertConfig.from_pretrained(model_path, num_labels=1, dropout=0.1, attention_dropout=0.1)
model = BertForSequenceClassification.from_pretrained(model_path, config = config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from torch import cuda
import torch
device = 'cuda' if cuda.is_available() else 'cpu'

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PearsonCorrelationLoss(nn.Module):
    def forward(self, preds, labels):
        preds = preds.squeeze()
        labels = labels.squeeze()
        
        x_mean = torch.mean(preds)
        y_mean = torch.mean(labels)
        cov = torch.sum((preds - x_mean) * (labels - y_mean))
        
        x_var = torch.sum((preds - x_mean) ** 2)
        y_var = torch.sum((labels - y_mean) ** 2)
        
        pearson_corr = cov / torch.sqrt(x_var * y_var)
        return 1 - pearson_corr

pearson_loss = PearsonCorrelationLoss()

from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = pearson_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [29]:
import numpy as np
from scipy.stats import pearsonr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
           }

In [43]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import torch

training_args = TrainingArguments(
    output_dir='/kaggle/working/model',
    report_to="none",
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=50,  # Evaluate every 2000 steps
    save_strategy="steps",
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    lr_scheduler_type='linear',  # Adjust learning rate scheduling
    learning_rate=4e-5,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [57]:
torch.cuda.empty_cache()

In [58]:
# results of normal bert on full data
train_results = trainer.train()

Step,Training Loss,Validation Loss,Pearson
50,0.918000,0.899605,0.074431
100,0.661700,0.969921,0.172455
150,0.497900,0.904321,0.193260
200,0.363700,0.793247,0.294838
250,0.385600,0.863126,0.399444
300,0.383800,0.793843,0.344691
350,0.362600,0.863155,0.290475
400,0.414600,0.877314,0.314699
450,0.381200,0.839101,0.350112
500,0.323200,0.987307,0.354541


In [31]:
# results of bert-medium on bible data
train_results = trainer.train()

Step,Training Loss,Validation Loss,Pearson
50,0.044900,0.032656,0.035876
100,0.031900,0.026587,0.016872
150,0.025300,0.019078,0.149549
200,0.020900,0.009597,0.349048
250,0.014800,0.008611,0.358525
300,0.010400,0.015509,0.321659
350,0.012600,0.008346,0.266406
400,0.010700,0.009879,0.226679
450,0.009900,0.006963,0.172984
500,0.009900,0.015630,0.250749
